In [1]:
from socket import *
import os
from pathlib import Path

In [ ]:
# import socket programming library 
import socket 

# import thread module 
from _thread import *
import threading 

print_lock = threading.Lock() 

# thread fuction 
def threaded(c,message): 
    while True: 
        typeOfRequest = message.split()[0]
        filename = message.split()[1]
        sendingport=message.split()[2]
        if typeOfRequest == 'GET':
            print ('File name is: ', filename)
            my_file = Path(filename)
            if my_file.is_file():
                f = open(filename)
                outputdata = f.read()
                c.send("HTTP/1.1 200 OK\r\n\r\n".encode())
                for i in range(0, len(outputdata)):  
                    c.send(outputdata[i].encode())
                c.send("\r\n".encode())
            elif typeOfRequest == 'POST':
                f = open(filename, "x")
                while message != '/r/n':
                    message =client.recv(1024).decode()
                    f.write(message)
                    #message =client.recv(1024).decode()
            c.close()
        else:
            c.send(("HTTP/1.1 400 Bad Request ETLAM!!\r\n\r\n").encode())

#        # data received from client 
#         data = c.recv(1024) 
#         if not data: 
#             print('Bye') 

#             # lock released on exit 
#             print_lock.release() 
#             break

#         # reverse the given string from client 
#         data = data[::-1] 

#         # send back reversed string to client 
#         c.send(data) 
#         c.send('200 OK'.encode('ascii'))
    # connection closed 
    c.close() 


def Main(): 
    host = "" 

    # reverse a port on your computer 
    # in our case it is 12345 but it 
    # can be anything 
    port = 12345
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
    s.bind((host, port)) 
    print("socket binded to post", port) 

    # put the socket into listening mode 
    s.listen(5) 
    print("socket is listening") 

    # a forever loop until client wants to exit 
    while True: 

        # establish connection with client 
        c, addr = s.accept() 
        print(addr[1])
        #gg = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
        #gg.bind((addr[0], addr[1])) 
        print("socket binded to post", port) 


        # lock acquired by client 
        print_lock.acquire() 
        message = c.recv(1024).decode()
        print('Connected to :', addr[0], ':', addr[1]) 
        print('message : '+message)
        start_new_thread(threaded, (c,message)) 

#         filename = message.split()[1]
#         typeOfRequest = message.split()[0]
#         sendingport=message.split()[2]
#         if typeOfRequest == 'GET':
#             print ('File name is: ', filename)
#             f = open(filename[0:])
#             outputdata = f.read()
#             c.send("HTTP/1.1 200 OK\r\n\r\n".encode())
#             for i in range(0, len(outputdata)):  
#                 c.send(outputdata[i].encode())
#             c.send("\r\n".encode())
#             c.close()
#         elif typeOfRequest == 'POST':
#             f = open(filename, "x")
#             while message != '/r/n':
#                 message =client.recv(1024).decode()
#                 f.write(message)
#                 #message =client.recv(1024).decode()

#         else:
#             c.send(("HTTP/1.1 400 Bad Request ETLAM!!\r\n\r\n").encode())
# 
         
        # Start a new thread and return its identifier 
        #start_new_thread(threaded, (c,))
        c.send('200 OK'.encode('ascii'))
    s.close() 


if __name__ == '__main__': 
    Main() 



socket binded to post 12345
socket is listening


In [6]:
print(os.listdir())

['Server.ipynb', '.ipynb_checkpoints']


In [6]:
import psutil
from psutil import process_iter
from signal import SIGTERM # or SIGKILL

for proc in process_iter():
    try:
        for conns in proc.connections(kind='inet'):
            if conns.laddr.port == 12345:
                    proc.send_signal(SIGTERM) # or SIGKILL
    except (psutil.ZombieProcess, psutil.AccessDenied, psutil.NoSuchProcess):   
        continue